<a href="https://colab.research.google.com/github/Miranda-Merino-Irigoyen-Jorge-Missael/Piloto/blob/main/Aprendiendo_Simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def carro(env): #Se usa la variable de entrada env como referencia del "enviroment" o "entorno"
    while True:
        print(f'Inicia el estacionado en: {env.now}') #env.now regresa el tiempo actual de la simulación
        duracion_aparcado = 5
        yield env.timeout(duracion_aparcado) #se crea el evento de estacionado/aparcado

        print(f'Inicia la conducción en: {env.now}')
        duracion_conduccion = 2
        yield env.timeout(duracion_conduccion) #se crea el evento de conducción

In [4]:
!pip install simpy
import simpy

In [7]:
env = simpy.Environment()
env.process(carro(env))

<Process(carro) object at 0x7dcab01f2cb0>

In [8]:
env.run(until=24)

Inicia el estacionado en: 0
Inicia la conducción en: 5
Inicia el estacionado en: 7
Inicia la conducción en: 12
Inicia el estacionado en: 14
Inicia la conducción en: 19
Inicia el estacionado en: 21


Ejemplo del itinerario de Juan:


In [5]:
class Juan:  # Definimos una clase llamada Juan.
    #Definimos el método '__init__' con 'env' como parametro.
    def __init__(self, env):
      #Hacemos que la variable 'env' (en este caso es el entorno de simulación) se convierta en un atributo de la clase 'Juan'
        self.env = env
        #Una vez declarado lo anterior podremos acceder al atributo env usando self.env
        #Creamos un atributo de la instancia llamado 'acción'#
        #Indicamos la acción de este proceso men el entorno y le indicamos que siga el #
        #flujo de eventos de la función self.rutina() #
        self.accion = env.process(self.rutina())

#Definimos un metodo describir la rutina de Juan#
    def rutina(self):
      #Declaramos el diccionario para pasar de números a día de la semana#
        semana = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes'}
        #Declaramos el bucle infinito para que la rutina se repita todos los días#
        while True:
          #Calculamos el día actual (self.env.now es el estado actual del entorno)
            dia = self.env.now // 24
            #Imprimimos el día de la semana de el tiempo donde se encuentre el entorno#
            print(f'Día {semana[dia]}')

            # Rutina: Antes de ir al trabajo#
            #Calculamos la hora actual del momento donde se encuentre el entorno#
            hora = self.env.now % 24
            print(f'A las {hora} horas Juan está en su casa')  # Imprimimos la hora en la que Juan está en casa.
            yield self.env.timeout(8)  # Hacemos que transcurran 8 horas, simulando que Juan pasa este tiempo en casa.

            # Llega al trabajo
            hora = self.env.now % 24  # Calculamos de nuevo la hora actual después de las 8 horas.
            print(f'A las {hora} horas Juan está en su trabajo')  # Imprimimos la hora en la que Juan llega al trabajo.

            # A continuación, se maneja la posibilidad de interrupciones mientras trabaja.#
            # Recuerde que la estructura 'try:' permite ejecutar codigo estando atento #
            # a ciertas interrumpciones #
            try:
                # Simulamos que Juan trabaja durante 8 horas.
                yield self.env.process(self.trabajar(8))  # Iniciamos el proceso `trabajar` que dura 8 horas.
            except simpy.Interrupt:  # Si ocurre una interrupción durante las horas de trabajo:
                # Juan tiene que ir a la junta de su hija
                dia = self.env.now // 24  # Calculamos el día de la interrupción.
                hora = self.env.now % 24  # Calculamos la hora de la interrupción.
                print(f'A las {hora} horas Juan va a la junta de su hija')  # Imprimimos que Juan va a la junta.
                yield self.env.timeout(2)  # Simulamos que la junta dura 2 horas.

            # Llega a la casa
            hora = self.env.now % 24  # Calculamos la hora actual después de la jornada laboral.
            print(f'A las {hora} horas Juan está en su casa\n')  # Imprimimos la hora en que Juan regresa a casa.
            yield self.env.timeout(24 - hora)  # Esperamos el tiempo restante para completar las 24 horas del día.
    #Defninimos el proceso de trabajar#
    def trabajar(self, duracion):  # Método que simula el tiempo de trabajo de Juan.
        yield self.env.timeout(duracion)  # Espera el tiempo especificado en `duracion` (en este caso, 8 horas).
#Defninimos el proceso de la junta#
def junta(env, persona):
  #Indicamos que se haga el proceso cuando el entorno este el martes a medio día#
    yield env.timeout(24+12)
    persona.accion.interrupt()

# Creamos el entorno de simulación 'env', que controlará el tiempo y los eventos de la simulación.
env = simpy.Environment()
# Creamos una instancia de la clase Juan, llamada 'persona', y le pasamos 'env' para que pueda interactuar con el entorno.
persona = Juan(env)
# Añadimos el proceso 'junta' al entorno de simulación 'env'. Este proceso representa una interrupción en la rutina de Juan.
# Le pasamos 'env' y 'persona' como argumentos para que pueda programar la interrupción en la rutina de Juan.
env.process(junta(env, persona))
# Ejecutamos la simulación con 'env.run()' y la configuramos para que corra hasta el tiempo 120 en horas.
# Durante este tiempo, el entorno avanzará y gestionará los eventos de 'persona' y 'junta'.
env.run(until=120)


Día Lunes
A las 0 horas Juan está en su casa
A las 8 horas Juan está en su trabajo
A las 16 horas Juan está en su casa

Día Martes
A las 0 horas Juan está en su casa
A las 8 horas Juan está en su trabajo
A las 12 horas Juan va a la junta de su hija
A las 14 horas Juan está en su casa

Día Miércoles
A las 0 horas Juan está en su casa
A las 8 horas Juan está en su trabajo
A las 16 horas Juan está en su casa

Día Jueves
A las 0 horas Juan está en su casa
A las 8 horas Juan está en su trabajo
A las 16 horas Juan está en su casa

Día Viernes
A las 0 horas Juan está en su casa
A las 8 horas Juan está en su trabajo
A las 16 horas Juan está en su casa



Otro ejemplo sobre una gasolinera:

In [13]:
import simpy
from random import random as random
from math import log

def gasolinera(env, bomba, num, ll, ls):
    for i in range(num): #El for es para delimitar en cuanto al número de carros que se van a atender
    #Se calcula el tiempo de llegada del carro usando #
    #una distribución exponencial inversa #
        llegada = -log(random())/ll
        #Pausamos la simulación hasta la llegada de un coche#
        yield env.timeout(llegada)
        #Imprimimos cuando llegue un coche#
        print(f'---> Ha llegado el carro {i} en el minuto {round(env.now, 2)}')
        #añadimos el prcoeso 'carro' al entorno#
        env.process(carro(env, bomba, i, ls))

def carro(env, bomba, num, ls):  # Esta función modela el proceso de carga de gasolina para un carro
    # Solicitud de carga de gasolina
    with bomba.request() as req:  # El carro solicita el recurso 'bomba' para poder cargar gasolina
        yield req  # El carro espera hasta que la bomba esté disponible

        # Se carga gasolina
        print(f'El carro {num} se comienza a cargar en el minuto {round(env.now, 2)}')  # Imprime el inicio de la carga, mostrando el minuto actual
        carga = -log(rd()) / ls  # Calcula el tiempo de carga (servicio) utilizando una distribución exponencial
        yield env.timeout(carga)  # Pausa la simulación durante el tiempo de carga calculado

        print(f'El carro {num} termina de cargar en el minuto {round(env.now, 2)}   --->')  # Imprime cuando la carga ha finalizado, mostrando el minuto actual
#Generamos el entorno#
entorno = simpy.Environment()
#Usando simpy.Resource() crearemos el objeto 'bomba' para que pueda ser usado#
bomba = simpy.Resource(entorno, capacity=1)
#Procesamos nuestra función de gasolinera en el entorno#
entorno.process(gasolinera(entorno, bomba, 7, 1/2, 1/3))
#Lo ejecutamos#
entorno.run()

---> Ha llegado el carro 0 en el minuto 0.3
El carro 0 se comienza a cargar en el minuto 0.3
---> Ha llegado el carro 1 en el minuto 0.57
---> Ha llegado el carro 2 en el minuto 1.53
El carro 0 termina de cargar en el minuto 2.65   --->
El carro 1 se comienza a cargar en el minuto 2.65
---> Ha llegado el carro 3 en el minuto 3.19
---> Ha llegado el carro 4 en el minuto 9.16
---> Ha llegado el carro 5 en el minuto 9.38
El carro 1 termina de cargar en el minuto 9.86   --->
El carro 2 se comienza a cargar en el minuto 9.86
---> Ha llegado el carro 6 en el minuto 13.12
El carro 2 termina de cargar en el minuto 18.95   --->
El carro 3 se comienza a cargar en el minuto 18.95
El carro 3 termina de cargar en el minuto 23.34   --->
El carro 4 se comienza a cargar en el minuto 23.34
El carro 4 termina de cargar en el minuto 26.13   --->
El carro 5 se comienza a cargar en el minuto 26.13
El carro 5 termina de cargar en el minuto 31.45   --->
El carro 6 se comienza a cargar en el minuto 31.45
El c